# Customer Churn Prediction with PySpark MLlib

In this lab, we will build a machine learning pipeline using PySpark's MLlib to predict customer churn.

We will use the [Telco Customer Churn](https://www.kaggle.com/datasets/blastchar/telco-customer-churn) dataset from Kaggle, which contains various features about customers that might influence churn.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, when
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, StandardScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Step 1: Initializing Spark Session

We start by initializing a Spark session.

In [ ]:
spark = SparkSession.builder \
    .appName("customer-churn-prediction") \
    .config("spark.sql.debug.maxToStringFields", 1000) \
    .getOrCreate()

## Step 2: Loading the Dataset

Next, we load the [Telco Customer Churn](https://www.kaggle.com/datasets/blastchar/telco-customer-churn) dataset from a CSV file.

This dataset includes various features such as customer demographics, account information, and services subscribed to by the customers.

In [ ]:
TELCO_DATA = "../data/telco/customer-churn.csv"

data = spark.read.csv(TELCO_DATA, header=True, inferSchema=True)
data.show(5)

## Step 3: Data Preprocessing

### 3.1 Handling Non-Numeric Values in `TotalCharges`

First, we need to address any non-numeric values in the **TotalCharges** column and convert them to a numerical type.
Additionally, we have to remove all rows with **null** values.

In [ ]:
data = data.withColumn("TotalCharges", 
                       when(regexp_replace(col("TotalCharges"), r"^[0-9]+(\.[0-9]+)?$", col("TotalCharges")).isNull(), None)
                       .otherwise(col("TotalCharges")))

data = data.withColumn("TotalCharges", col("TotalCharges").cast("double"))
data = data.dropna(subset=["TotalCharges"])


data.printSchema()

### 3.2 Handling Categorical Features

We use `StringIndexer` to convert categorical variables into numerical format. This includes features like: 

<div style="display: flex; justify-content: space-between;">

<div style="flex: 1; padding: 10px;">
  <ul>
    <li>gender</li>
    <li>Partner</li>
    <li>Dependents</li>
    <li>PhoneService</li>
    <li>MultipleLines</li>
  </ul>
</div>

<div style="flex: 1; padding: 10px;">
  <ul>
    <li>InternetService</li>
    <li>OnlineSecurity</li>
    <li>OnlineBackup</li>
    <li>DeviceProtection</li>
    <li>TechSupport</li>
  </ul>
</div>

<div style="flex: 1; padding: 10px;">
  <ul>
    <li>StreamingTV</li>
    <li>StreamingMovies</li>
    <li>Contract</li>
    <li>PaperlessBilling</li>
    <li>PaymentMethod</li>
  </ul>
</div>

</div>

To illustrate the preprocessing, let's take the `gender` feature as an example. 

| gender | genderIndexed |
|--------|---------------|
| Female | 0             |
| Male   | 1             |

In [ ]:
indexer = StringIndexer(
    inputCols=[
    'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 
    'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod'], 
    outputCols=[
    'genderIndexed', 'PartnerIndexed', 'DependentsIndexed', 'PhoneServiceIndexed', 'MultipleLinesIndexed', 
    'InternetServiceIndexed', 'OnlineSecurityIndexed', 'OnlineBackupIndexed', 'DeviceProtectionIndexed', 
    'TechSupportIndexed', 'StreamingTVIndexed', 'StreamingMoviesIndexed', 'ContractIndexed', 
    'PaperlessBillingIndexed', 'PaymentMethodIndexed']
)

### 3.3 Assembling Features

We use `VectorAssembler` to combine all feature columns into a single vector column named `features`.

For each row, the values from *MonthlyCharges*, *TotalCharges*, *genderIndexed*, and others are concatenated in order to form the feature vector.


#### Why Use VectorAssembler?

- **Uniform Input Format:** Machine learning algorithms in Spark MLlib require input data to be in the form of a single vector of features. VectorAssembler helps in combining multiple feature columns into this required format.

- **Pipeline Compatibility:** It ensures compatibility with the Spark MLlib pipeline, where all feature transformations are applied in sequence before training the model.

- **Efficiency:** It consolidates the feature columns into a single vector, which can be processed more efficiently by Spark's distributed computing framework.


| id |features             |
|----|---------------------|
| 0  |[18.0,1.0,0.0,5000.0]|
| 1  |[20.0,0.0,1.0,6000.0]|
| 2  |[30.0,1.0,0.0,7000.0]|
| 3  |[40.0,0.0,1.0,8000.0]|

In [ ]:
assembler = VectorAssembler(
    inputCols=[
        'SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'genderIndexed', 'PartnerIndexed', 
        'DependentsIndexed', 'PhoneServiceIndexed', 'MultipleLinesIndexed', 'InternetServiceIndexed', 
        'OnlineSecurityIndexed', 'OnlineBackupIndexed', 'DeviceProtectionIndexed', 'TechSupportIndexed', 
        'StreamingTVIndexed', 'StreamingMoviesIndexed', 'ContractIndexed', 'PaperlessBillingIndexed', 
        'PaymentMethodIndexed'],
    outputCol='features'
)

### 3.4 Standardizing Features

We use `StandardScaler` to standardize the feature values.

Standardization is a preprocessing step in machine learning where features are rescaled so that they have the properties of a standard normal distribution with a mean of zero and a standard deviation of one.

<div style="text-align: center;">
    <img src="images/normalization-standardization.jpg" alt="Normalization vs Standardization" width="848" height="477">
</div>

#### Why Standardize?

- **Normalization:** Ensures that all features contribute equally to the model, preventing features with larger scales from dominating the learning process.

- **Performance:** Many machine learning algorithms (e.g., linear regression, logistic regression, k-means) perform better or converge faster when features are on a similar scale.

- **Stability:** Helps in numerical stability, particularly for gradient-based optimization algorithms.

#### How Features are Standardized

The StandardScaler in PySpark MLlib standardizes features by subtracting the mean and dividing by the standard deviation of each feature. Mathematically, for a feature $x$:

$z = \frac{x - \mu}{\sigma}$

where:
- $x$ is the original feature value.
- $\mu$ is the mean of the feature.
- $\sigma$ is the standard deviation of the feature.
- $z$ is the standardized feature value.

In [ ]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

### 3.5 Selecting the Target Variable or Label

We select the target variable, which is `Churn` in this case.
We ensure that only rows with Churn values of Yes or No are kept.
Then, we convert the Churn column to a binary format where Yes is mapped to 1.0 and No is mapped to 0.0.

In [ ]:
data = data.filter((col('Churn') == 'Yes') | (col('Churn') == 'No'))
data = data.withColumn('label', when(col('Churn') == "Yes", 1.0).otherwise(0.0))

## Step 4: Splitting the Data

We split the dataset into training (80%) and test (20%) sets to allow for model training and evaluation.

In [ ]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

## Step 5: Initializing the RandomForestClassifier

We initialize a random forest classifier, which will be used to predict customer churn.

In [ ]:
classifier = RandomForestClassifier(featuresCol='scaledFeatures', labelCol='label')

## Step 6: Creating a Parameter Grid

We create a parameter grid for hyperparameter tuning, allowing us to experiment with different values for the number of trees and the maximum depth of the trees.

In [ ]:
param_grid = ParamGridBuilder() \
    .addGrid(classifier.numTrees, [10, 20, 30]) \
    .addGrid(classifier.maxDepth, [5, 10, 15]) \
    .build()

## Step 7: Initializing the Evaluator

To evaluate the performance of our binary classification model, we use the ROC curve and the area under the curve (AUC) as our evaluation metrics.


- **[ROC Curve](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc):**
The Receiver Operating Characteristic (ROC) curve is a graphical representation of a classifier's performance across all classification thresholds. It plots the True Positive Rate (TPR) against the False Positive Rate (FPR) at various threshold settings.

- **AUC (Area Under the Curve):**
The AUC is a single scalar value that summarizes the performance of the classifier. It represents the probability that a randomly chosen positive instance is ranked higher than a randomly chosen negative instance. An AUC value of 1 indicates a perfect model, while an AUC value of 0.5 suggests a model with no discriminative ability.

- **BinaryClassificationEvaluator:**
In Spark MLlib, the BinaryClassificationEvaluator is used to compute the AUC for binary classification models. It evaluates the model's performance based on the area under the ROC curve (AUC).

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol='label', rawPredictionCol='prediction')

## Step 8: Initializing the CrossValidator

We use cross-validation to perform hyperparameter tuning and model evaluation.

K-fold cross validation performs model selection by splitting the dataset into a set of non-overlapping randomly partitioned folds which are used as separate training and test datasets.

With k=5 folds, K-fold cross validation will generate 5 (training, test) dataset pairs, each of which uses 4/5 of the data for training and 1/5 for testing.

In [ ]:
crossval = CrossValidator(estimator=classifier,
                          estimatorParamMaps=param_grid,
                          evaluator=evaluator,
                          numFolds=5)

## Step 9: Creating and Fitting the Pipeline

We create a pipeline that includes the preprocessing steps, the random forest classifier, and the cross-validator. The model is then fitted to the training data.

In [ ]:
pipeline = Pipeline(stages=[indexer, assembler, scaler, crossval])

model = pipeline.fit(train_data)

## Step 10: Making Predictions and Evaluating the Model

We make predictions on the test set and evaluate the model using the AUC metric.

In [ ]:
predictions = model.transform(test_data)

auc = evaluator.evaluate(predictions)
print(f"AUC: {auc}")

## Step 11: Displaying Feature Importance

We extract and plot the feature importances to understand which features are most influential in predicting customer churn.

In [ ]:
rf_model = model.stages[-1].bestModel
feature_importances = rf_model.featureImportances.toArray()

features = ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'genderIndexed', 'PartnerIndexed', 
            'DependentsIndexed', 'PhoneServiceIndexed', 'MultipleLinesIndexed', 'InternetServiceIndexed', 
            'OnlineSecurityIndexed', 'OnlineBackupIndexed', 'DeviceProtectionIndexed', 'TechSupportIndexed', 
            'StreamingTVIndexed', 'StreamingMoviesIndexed', 'ContractIndexed', 'PaperlessBillingIndexed', 
            'PaymentMethodIndexed']

feature_importances_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importances
})

feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importances_df)
plt.xlabel('Feature Importance', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.title('Feature Importance for Customer Churn Prediction', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

## Step 12: Closing the Spark Session

Finally, we close the Spark session to release resources.

In [ ]:
spark.stop()